In [1]:
import pandas as pd
import os

In [ ]:
conda install -c conda-forge seaborn

In [2]:
# conda install -c conda-forge seaborn
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
data_path = r"../data/original/DWD/annual/kl/historical/"
station_file_name = r"KL_Jahreswerte_Beschreibung_Stationen.txt"

#print(f"Data Path: {data_path:s}") 

In [ ]:
full_fname = data_path + station_file_name
print(full_fname)

In [ ]:
#fh = open(full_fanme)
headers = open(full_fname).readline().split()
df_stat = pd.read_fwf(
    full_fname,
    encoding="cp1252",
    skiprows=2,
    names=headers,
    index_col="Stations_id", 
    parse_dates=["von_datum","bis_datum"])

In [ ]:
idx_NRW    = df_stat["Bundesland"] == "Nordrhein-Westfalen"
idx_active = df_stat["bis_datum"]  == "2020-12-31"
idx_early  = df_stat["von_datum"]   < "1930"

In [ ]:
df_stat_short = df_stat[idx_NRW & idx_active & idx_early]

In [ ]:
df_stat_short

In [ ]:
fname_list = os.listdir(data_path)

In [ ]:
f_dict={}

for fname in fname_list:
    try:
        key=fname.split("_")[2]
        f_dict[int(key)]=fname
    except:
        print(f"Cannot convert \'{key:s}\' to integer")

n=5
print("Frist {n:d} items of f_dict:")
print({k: f_dict[k] for k in list(f_dict)[:5]})


In [ ]:
zip_list=[]
for k in df_stat_short.index:
    zip_list.append(data_path + f_dict[k])

In [ ]:
# MORE ELEGANT!
zip_list = [data_path + f_dict[k] for k in df_stat_short.index]

In [ ]:
zip_list

In [ ]:
import pandas as pd
from zipfile import ZipFile

# Very compact code.
df = pd.DataFrame()
for elt in zip_list:
    with ZipFile(elt) as myzip:
        
        # read the time series data from the file starting with "produkt"
        prodfilename = [name for name in myzip.namelist() if name.split("_")[0]=="produkt"][0] 
        #print(prodfilename)
        
        # open just the produkt file within the archive
        with myzip.open(prodfilename) as myfile:
            dftmp = pd.read_csv(myfile, sep =";", parse_dates = ["MESS_DATUM_BEGINN", "MESS_DATUM_ENDE"], index_col = "MESS_DATUM_BEGINN", na_values = [-999.0])
            # select the series with the average annual temp ("JA_TT"), rename the series with the station_id and make it a df.
            s = dftmp["JA_TT"].rename(dftmp["STATIONS_ID"][0]).to_frame()
            # outer merge.
            df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

#df.index.names = ["year"]
df.index.rename(name = "year", inplace = True)
df

In [ ]:
# Replace full datetime with year as integer
df.set_index(df.index.year, inplace= True) # extract year from index as int

In [ ]:
df.head()

In [ ]:
mean = df[(df.index >= 1961) & (df.index <= 1990)].mean() # mean annual temp between 1961 and 1990
mean

In [ ]:
df_temp_diff = (df - mean)

In [ ]:
df_temp_diff

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

In [ ]:
# plot
sns.set_style('ticks')
fig1, ax1 = plt.subplots(dpi = 300, figsize = (30,10))

# sns.heatmap(df_temp_diff.T, cmap='coolwarm', vmin = -2, vmax = 2, ax = ax4)
sns.heatmap(df.T, cmap='coolwarm', ax = ax1)

fig1.savefig('NRW_Annual_Temp_Stripes_01.png')


In [ ]:
# plot
sns.set_style('ticks')
fig2, ax2 = plt.subplots(dpi = 150, figsize = (12,4))

#sns.heatmap(df_tdiff, cmap='coolwarm', annot=True, vmin = -2, vmax = 2, ax = ax3)
sns.heatmap(df[df.index >= 1930].T, cmap='coolwarm', ax = ax2)
#ax4.set_xticklabels(df.index.strftime('%Y'))

fig2.savefig('NRW_Annual_Temp_Stripes_02.png')

In [ ]:
# plot
sns.set_style('ticks')
fig3, ax3 = plt.subplots(dpi = 150, figsize = (12,4))

#sns.heatmap(df_tdiff, cmap='coolwarm', annot=True, vmin = -2, vmax = 2, ax = ax3)
sns.heatmap(df_temp_diff[df_temp_diff.index >= 1930].T, cmap='coolwarm', vmin = -2, vmax = 2, ax = ax3)
#ax4.set_xticklabels(df.index.strftime('%Y'))

fig3.savefig('NRW_Annual_Temp_Diff_Stripes_02.png')

In [ ]:
df_stat_short["Stationsname"]

In [ ]:
df_stat_short["von_datum"]

In [ ]:
df.head()